# Map points to drainage basins and ice sheets
This notebook provides a mapping between points for which SNOWPACK was run and the drainage basins and ice sheets they belong to.

In [ ]:
import re
import numpy as np
import fiona
import pyproj

from shapely.geometry import Polygon, Point, shape
from shapely.ops import transform
import shapely.prepared

# Initialize required projections
wgs84 = pyproj.CRS('EPSG:4326')  # WGS 84
aps = pyproj.CRS('EPSG:3031')    # Antarctic Polar Stereographic
project_to_aps = pyproj.Transformer.from_crs(wgs84, aps, always_xy=True).transform

## Read SNOWPACK points
SNOWPACK.pts should contain `latitude longitude` for all SNOWPACK run points

In [ ]:
# Read points for which we performed SNOWPACK simulations
SP_pts = np.loadtxt("SNOWPACK.pts", comments="#", delimiter=" ", unpack=False)

## Read in drainage basins and icesheet boundaries
These files were obtained from http://imbie.org/imbie-3/drainage-basins/ and https://earth.gsfc.nasa.gov/cryo/data/polar-altimetry/antarctic-and-greenland-drainage-systems.

In [ ]:
# Rignot
ais_basins_shapefile="./rignot/ANT_Basins_IMBIE2_v1.6.shp"                            # in Antarctic Polar Stereographic
ais_icesheets_shapefile="./rignot/ANT_IceSheets_IMBIE2/ANT_IceSheets_IMBIE2_v1.6.shp" # in Antarctic Polar Stereographic
gris_basins_shapefile="./rignot/GRE_Basins_IMBIE2_v1.3.shp"                           # in EPSG:4326
gris_icesheets_shapefile="./rignot/GRE_IceSheet_IMBIE2/GRE_IceSheet_IMBIE2_v1.shp"    # in EPSG:4326

# Zwally
ais_basins_shapefile_zwally="./zwally/ant_grounded_drainagesystem_polygons.txt"       # in EPSG:4326
ais_icesheets_shapefile_zwally="./zwally/Ant_Grounded_IceSheets_Polygon-1.txt"        # in EPSG:4326
gris_basins_shapefile_zwally="./zwally/grndrainagesystems_ekholm.txt"                 # in EPSG:4326
gris_icesheets_shapefile_zwally="./zwally/GrnIceSheet_Ekholm-1.txt"                   # in EPSG:4326

# Mappings:
AIS = ["None" for i in range(31)]
AIS[28]="AP"
AIS[29]="WAIS"
AIS[30]="EAIS"
# Antarctica basins in Zwally labeled 1 thru 27
AB_labels_zw = [i for i in range(1,28)]
# Greenland basins in Zwally labeled as follows (in order they appear in the file):
GB_labels_zw = [1.1,1.2,1.3,1.4,2.1,2.2,3.1,3.2,3.3,4.1,4.2,4.3,5.0,6.1,6.2,7.1,7.2,8.1,8.2]

### Read in Zwally delineations into shapely Polygons

In [ ]:
#
# Antarctica (note, we convert polygons to EPSG:3031, to be able to properly use .contain())
# Since the South Pole is contained in the polygons, the polygon wraps around the South Pole.
#

# Ice sheets
AIS_pl_zw = ["None" for i in range(31)]   # Initialize array
for i in range(28,31):                    # 28-30: See mappings above
    f = open(ais_icesheets_shapefile_zwally, "r")
    # Skip header
    for l in f:
        if l == "END OF HEADER\n":
            break
    # Read polygon
    pl = []
    for l in f:
        l_s = re.split(' +', l.replace("\n", "").replace("\t", " ").lstrip().rstrip())
        if len(l_s)==3 and int(l_s[2])==i:
            pt = Point(float(l_s[1]), float(l_s[0]))   # WGS-84, EPSG:4326
            pt_aps = transform(project_to_aps, pt)     # Antarctic Polar Stereographic, EPSG: 3031
            pl.append([pt_aps.x, pt_aps.y])
    f.close()
    AIS_pl_zw[i]=shapely.prepared.prep(Polygon(pl))

# Drainage basins
AB_pl_zw = ["None" for i in range(28)]   # Initialize array
for i in range(1,28):                    # Drainage basins IDs range from 1-27
    f = open(ais_basins_shapefile_zwally, "r")
    # Skip header
    for l in f:
        if l == "END OF HEADER\n":
            break
    # Read polygon
    pl = []
    for l in f:
        l_s = re.split(' +', l.replace("\n", "").replace("\t", " ").lstrip().rstrip())
        if len(l_s)==3 and int(l_s[2])==i:
            pt = Point(float(l_s[1]), float(l_s[0]))   # WGS-84, EPSG:4326
            pt_aps = transform(project_to_aps, pt)     # Antarctic Polar Stereographic, EPSG: 3031
            pl.append([pt_aps.x, pt_aps.y])
    f.close()
    AB_pl_zw[i]=shapely.prepared.prep(Polygon(pl))

In [ ]:
#
# Greenland
#

# Ice sheets
GrIS_pl_zw = "None"
f = open(gris_icesheets_shapefile_zwally, "r")
pl = []
for l in f:
    # Skip header
    if l == "END OF HEADER\n":
        break
# Read polygon
for l in f:
    l_s = re.split(' +', l.replace("\n", "").replace("\t", " ").lstrip().rstrip())
    if len(l_s)==3 and int(l_s[0])==9:
        pl.append([float(l_s[2])-360., float(l_s[1])])
f.close()
GrIS_pl_zw=shapely.prepared.prep(Polygon(pl))

# Drainage basins
GB_pl_zw = ["None" for i in range(len(GB_labels_zw))]   # Initialize array
for i in range(len(GB_labels_zw)):                      # Loop over label array
    f = open(gris_basins_shapefile_zwally, "r")
    # Skip header
    for l in f:
        if l == "END OF HEADER\n":
            break
    # Read polygon
    pl = []
    for l in f:
        l_s = re.split(' +', l.replace("\n", "").replace("\t", " ").lstrip().rstrip())
        if len(l_s)==3 and float(l_s[0])==GB_labels_zw[i]:
            pl.append([float(l_s[2])-360., float(l_s[1])])
    f.close()
    GB_pl_zw[i]=shapely.prepared.prep(Polygon(pl))

## Loop over SNOWPACK points and list in which drainage basin and ice sheet they are located

In [ ]:
# Print header
print("#lat lon Ant/Gr Ant_IS_R Ant_ISSub_R Ant_Basin_R Ant_BasinSub_R Gr_IS_R Gr_Basin_R, Ant_IS_Z Ant_ISSub_Z Ant_Basin_Z Ant_BasinSub_Z Gr_IS_Z Gr_Basin_Z")
print("# *_R = Rignot, *_Z = Zwally")
# Loop over SNOWPACK points
for i in SP_pts:
    pt = Point(i[1], i[0])                  # WGS-84, EPSG:4326
    pt_aps = transform(project_to_aps, pt)  # Antarctic Polar Stereographic, EPSG: 3031


    #
    # Rignot
    #

    # Initialize info for Antarctica
    Ant = False
    regAB = None      # Region Antarctic Basins
    subregAB = None   # Subregion Antarctic Basins
    regAIS = None     # Region Antarctic Icesheet
    subregAIS = None  # Subregion Antarctic Icesheet
    # Initialize info for Greenland
    Gr = False
    regGB = None      # Region Greenland Basins
    regGIS = None     # Region Greenland Icesheet

    # Check Antarctica
    # Icesheets
    for f in fiona.open(ais_icesheets_shapefile):
        g=shape(f['geometry'])
        if g.contains(pt_aps):
            Ant = True
            regAIS=f['properties']['Regions']
            subregAIS=f['properties']['Subregion']
    # Drainage Basins
    for f in fiona.open(ais_basins_shapefile):
        g=shape(f['geometry'])
        if g.contains(pt_aps):
            Ant = True
            regAB = f['properties']['Regions']
            subregAB = f['properties']['Subregion']

    # Check Greenland
    # Icesheets
    for f in fiona.open(gris_icesheets_shapefile):
        g=shape(f['geometry'])
        if g.contains(pt):
            Gr = True
            regGIS = f['properties']['SUBREGION1']
    # Drainage Basins
    for f in fiona.open(gris_basins_shapefile):
        g=shape(f['geometry'])
        if g.contains(pt):
            Gr = True
            regGB = f['properties']['SUBREGION1']

    # print info
    print(i[0], i[1], end = '')
    if Ant:
        print(' Ant', end = '')
    elif Gr:
        print(' Gr', end = '')
    else:
        print(' None', end = '')
    print('', regAIS, subregAIS, regAB, subregAB, regGIS, regGB, end = '')


    #
    # Zwally
    #

    # Initialize variables for Antarctica
    Ant = False
    regAB = None      # Region Antarctic Basins
    subregAB = None   # Subregion Antarctic Basins
    regAIS = None     # Region Antarctic Icesheet
    subregAIS = None  # Subregion Antarctic Icesheet
    # Initialize variables for Greenland
    Gr = False
    regGB = None      # Region Greenland Basins
    regGIS = None     # Region Greenland Icesheet

    # Check Antarctica
    # Icesheets
    for n in range(28, 31):  # Loop over the Antarctic Icesheets
        if AIS_pl_zw[n].contains(pt_aps):
            Ant = True
            regAIS=AIS[n]
            subregAIS="None"
    # Drainage Basins
    for n in range(1,28):    # Loop over the Antarctic Drainage Basins
        if AB_pl_zw[n].contains(pt_aps):
            Ant = True
            regAIS=n
            subregAIS="None"

    # Check Greenland
    # Icesheets
    if GrIS_pl_zw.contains(pt):
        Gr = True
        regGIS = "GrIS"
    # Drainage Basins
    for n in range(len(GB_labels_zw)):              # Loop over label array
        if GB_pl_zw[n].contains(pt):
            Gr = True
            regGB=GB_labels_zw[n]

    print('', regAIS, subregAIS, regAB, subregAB, regGIS, regGB)